In [1]:
import pygame, sys, random
import numpy as np
import math
import matplotlib.pyplot as plt
import pylab
from RVO_Py_MAS.RVO import RVO_update, reach, compute_V_des
from RVO_Py_MAS.vis import visualize_traj_dynamic
import time

num_episodes = 250

obstacleRadius = 10
agentRadius = 10

# get size of state and action from environment

boundaryPos = [100, 100]
boundaryLength = [70,70]
boundaryRadius = 40
dispSize = [1280, 960]
initPosAgentStandard = [dispSize[0] - 100, 100]
initPosAgent = initPosAgentStandard
goalPos = [100, 900]
rangeNumber = 5
obsNumber = 100

In [2]:
ws_model = dict()
ws_model['robot_radius'] = agentRadius
ws_model['circular_obstacles'] = []
ws_model['boundary'] = []

In [3]:
def rangeFinder(allObsPos, rangeCenter):
    indexMin = []
    tmpDist = [0 for _ in range(obsNumber)]
    allObsAgtDistance = [0 for _ in range(obsNumber)]
    for i in range(0, obsNumber):
        allObsAgtDistance[i] = math.sqrt((allObsPos[i][0] - rangeCenter[0])**2 + (allObsPos[i][1] - rangeCenter[1])**2)
        tmpDist[i] = allObsAgtDistance[i]
    tmpDist.sort()
    for i in range(0, rangeNumber):
        indexMin.append(allObsAgtDistance.index(tmpDist[i]))
    return indexMin

In [4]:
#### pygame.init()
screen = pygame.display.set_mode(dispSize)
screen.fill([220, 220, 220])

rList, episodes = [], []
fps = 0

# Make Obstacles (obsNumber)
obstaclePos = [[0, 0] for _ in range(obsNumber)]
# for i in range(0,obsNumber):
#     obstaclePos[i][0] = int(150 + i % 7 * 150)
#     obstaclePos[i][1] = int(150 + i / 7 * 150)
# Number of Obstacles = 100
for i in range(0,obsNumber):
    obstaclePos[i][0] = int(150 + i % 10 * 100)
    obstaclePos[i][1] = int(80 + i / 10 * 100)
# Number of Obstacles = 150
# Number of Obstacles = 200
# for i in range(0,obsNumber):
#     obstaclePos[i][0] = int(150 + i % 14 * 80)
#     obstaclePos[i][1] = int(80 + i / 14 * 80)

index = rangeFinder(obstaclePos, initPosAgent)
rangeObstacle = [[0,0] for _ in range(rangeNumber)]
for j in range(0, rangeNumber):
    rangeObstacle[j][0] = obstaclePos[index[j]][0]
    rangeObstacle[j][1] = obstaclePos[index[j]][1]

for e in range(num_episodes):
    # Initialize
#     start = time.time()
#     frame = 0
    done = False
    score = 0
    x = initPosAgent[0]
    y = initPosAgent[1]
    
    X = [initPosAgent] + rangeObstacle
    V = [[0,0] for i in xrange(len(X))]
    V_max = [2.0 for i in xrange(len(X))]
#     goal = [goalPos] + rangeObstacle
    randomNo = [0 for _ in range(rangeNumber)]
    goal = [goalPos]
    for i in range(0, rangeNumber):
        randomNo[i] = random.randrange(0, 100)
        goal += [initPosAgent]

    print("Episode ", e, "Starts!")

    while not done:
#         frame += 1
        V_des = compute_V_des(X, goal, V_max)
        print V_des
        V = RVO_update(X, V_des, V, ws_model)
        print V
        print "================================================"
        x += int(V[0][0])
        y += int(V[0][1])
        
        collisionFlag = 0
        pygame.draw.circle(screen, [100, 100, 255], [x,y], 10, 0)
        
        if math.sqrt((x -  goalPos[0])**2 + (y - goalPos[1])**2) <= agentRadius * 2:
            print("Goal Reached!")
            collisionFlag = 1
            done = True
        index = rangeFinder(obstaclePos, [x,y])
        for j in range(0, rangeNumber):
#             V[j + 1][0] = random.randrange(-1, 2)
#             V[j + 1][1] = random.randrange(-1, 2)
            rangeObstacle[j][0] = obstaclePos[index[j]][0] + int(round(V[j + 1][0]))
            rangeObstacle[j][1] = obstaclePos[index[j]][1] + int(round(V[j + 1][1]))
            obstaclePos[index[j]][0] = rangeObstacle[j][0]
            obstaclePos[index[j]][1] = rangeObstacle[j][1]
            
            if math.sqrt((x - rangeObstacle[j][0])**2 + (y - rangeObstacle[j][1])**2) < 19:
                print("Collision!")
                collisionFlag = -1
                done = True
        k = 0
        for i in range(0,obsNumber):
            if i == index[0] or i == index[1] or i == index[2] or i == index[3] or i == index[4]:
                pygame.draw.circle(screen, [255, 255, 50], rangeObstacle[k], obstacleRadius, 0)
                k += 1
            else:
                pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], obstacleRadius, 0)
                obstaclePos[i][0] = obstaclePos[i][0] + random.randrange(-1,2)
                obstaclePos[i][1] = obstaclePos[i][1] + random.randrange(-1,2)
        goal = [goalPos]
        for i in range(0, rangeNumber):
            goal += [[rangeObstacle[i][0] + random.randrange(-1,2), rangeObstacle[i][1] + random.randrange(-1,2)]]
#             if i == rangeNumber - 1:
#                 goal += [rangeObstacle[0]]
#             else:
#                 goal += [rangeObstacle[i + 1]]
        X = [[x, y]] + rangeObstacle
#         goal = [goalPos, obstaclePos[index[0]], obstaclePos[index[1]], obstaclePos[index[2]], obstaclePos[index[3]], obstaclePos[index[4]]]

        if done:
            if collisionFlag == 1:
                rList.append(1)
            elif collisionFlag == -1:
                rList.append(0)
            # every episode, plot the play time
            initPosAgent = initPosAgentStandard
            obstaclePos = [[0, 0] for _ in range(obsNumber)]
            # Number of Obstacles = 50
#             for i in range(0,obsNumber):
#                 obstaclePos[i][0] = int(150 + i % 7 * 150)
#                 obstaclePos[i][1] = int(150 + i / 7 * 150)
            # Number of Obstacles = 100
            for i in range(0,obsNumber):
                obstaclePos[i][0] = int(150 + i % 10 * 100)
                obstaclePos[i][1] = int(80 + i / 10 * 100)
            # Number of Obstacles = 150
#             for i in range(0,obsNumber):
#                 obstaclePos[i][0] = int(150 + i % 12 * 100)
#                 obstaclePos[i][1] = int(80 + i / 12 * 100)
            # Number of Obstacles = 200
#             for i in range(0,obsNumber):
#                 obstaclePos[i][0] = int(150 + i % 14 * 80)
#                 obstaclePos[i][1] = int(80 + i / 14 * 80)

            episodes.append(e)
            
        pygame.draw.circle(screen, [100,255,100], goalPos, 10, 0)
        pygame.display.flip()
        screen.fill([220,220,220])
#         final = time.time()
#         elapsed = final - start
#         if frame >= 10:
#             break
#     fps += frame / elapsed
#     print("Frame Per Second: " + str(frame / elapsed))
    
    print("Percent of successful episodes: " + str(100.0 * sum(rList)/(e + 1)) + "%")
print("fps average: " + str(fps / (e + 1)))

('Episode ', 0, 'Starts!')
[[-1.6071131909419312, 1.1904542155125417], [1.9767283665137618, 0.3041120563867326], [1.7033054746839078, -1.0481879844208664], [1.1709758338787974, -1.6213511546014119], [1.9924725460347519, 0.17325848226389148], [1.8889858362772252, -0.6570385517486]]
[[-1.607297888266654, 1.1902048310704878], [0.0, 1.9980019980019983], [-1.4132142689802296, -1.4132142689802296], [-1.4132142689802296, 1.4132142689802296], [-1.4132142689802296, -1.4132142689802296], [-1.4132142689802296, 1.4132142689802296]]
[[-1.6067698691864563, 1.1909175561038763], [0, 0], [-1.4132142689802296, -1.4132142689802296], [-1.4132142689802296, -1.4132142689802296], [-1.4132142689802296, -1.4132142689802296], [1.4132142689802296, -1.4132142689802296]]
[[-1.606954583686244, 1.1906682987758799], [-1.4132142689802296, 1.4132142689802296], [0.0, 1.9980019980019983], [1.4132142689802296, -1.4132142689802296], [1.4132142689802296, -1.4132142689802296], [-1.4132142689802296, 1.4132142689802296]]
[[-1.

[[-1.614551185653917, 1.1803466434926209], [0.0, 1.9980019980019983], [0, 0], [1.9980019980019983, 0.0], [-1.4132142689802296, -1.4132142689802296], [1.4132142689802296, -1.4132142689802296]]
[[-1.6147522252213184, 1.1800715970519255], [1.4132142689802296, 1.4132142689802296], [1.4132142689802296, -1.4132142689802296], [0, 0], [-1.4132142689802296, -1.4132142689802296], [1.9980019980019983, 0.0]]
[[-1.614953659119211, 1.1797959124499295], [1.9980019980019983, 0.0], [1.9980019980019983, 0.0], [-1.9980019980019983, 0.0], [1.9980019980019983, 0.0], [0.0, -1.9980019980019983]]
[[-1.6151554884747195, 1.1795195874937094], [-1.4132142689802296, -1.4132142689802296], [0.0, 1.9980019980019983], [1.4132142689802296, 1.4132142689802296], [1.4132142689802296, 1.4132142689802296], [-1.4132142689802296, 1.4132142689802296]]
[[-1.6153577144191025, 1.179242619980417], [0.0, -1.9980019980019983], [1.4132142689802296, -1.4132142689802296], [1.4132142689802296, 1.4132142689802296], [0.0, 1.99800199800199

[[-1.6236623856018644, 1.1677815937136502], [-1.4132142689802296, -1.4132142689802296], [0.0, -1.9980019980019983], [1.4132142689802296, 1.4132142689802296], [1.4132142689802296, 1.4132142689802296], [-1.9980019980019983, 0.0]]
[[-1.6238824434112038, 1.1674755652223114], [-1.4132142689802296, 1.4132142689802296], [1.4132142689802296, -1.4132142689802296], [0.0, -1.9980019980019983], [0, 0], [0, 0]]
[[-1.6241029518127972, 1.1671687880000226], [1.9980019980019983, 0.0], [-1.9980019980019983, 0.0], [-1.4132142689802296, 1.4132142689802296], [-1.4132142689802296, 1.4132142689802296], [0.0, -1.9980019980019983]]
[[-1.6235385462018865, 1.167953750073806], [0.0, 1.9980019980019983], [-1.9980019980019983, 0.0], [1.9980019980019983, 0.0], [1.4132142689802296, -1.4132142689802296], [1.9980019980019983, 0.0]]
[[-1.6237591507316738, 1.1676470297396304], [0.0, 1.9980019980019983], [1.9980019980019983, 0.0], [1.9980019980019983, 0.0], [1.9980019980019983, 0.0], [0.0, 1.9980019980019983]]
[[-1.623980

[[-1.6320475393499487, 1.1560336737062138], [0.0, 1.9980019980019983], [0.0, -1.9980019980019983], [-1.4132142689802296, -1.4132142689802296], [-1.4132142689802296, 1.4132142689802296], [1.4132142689802296, -1.4132142689802296]]
[[-1.6322876708416734, 1.1556945862002008], [0, 0], [-1.4132142689802296, -1.4132142689802296], [0.0, 1.9980019980019983], [1.4132142689802296, 1.4132142689802296], [-1.4132142689802296, -1.4132142689802296]]
[[-1.6325283167155922, 1.1553546224293432], [0.0, -1.9980019980019983], [0.0, 1.9980019980019983], [0.0, 1.9980019980019983], [1.4132142689802296, 1.4132142689802296], [0, 0]]
[[-1.632769478563495, 1.1550137790481314], [1.4132142689802296, -1.4132142689802296], [-1.4132142689802296, -1.4132142689802296], [-1.4132142689802296, 1.4132142689802296], [1.4132142689802296, -1.4132142689802296], [-1.9980019980019983, 0.0]]
[[-1.6330111579834008, 1.1546720526942715], [-1.4132142689802296, 1.4132142689802296], [1.9980019980019983, 0.0], [-1.4132142689802296, 1.4132

[[-1.6372487478804638, 1.1486553715422623], [-1.4132142689802296, 1.4132142689802296], [1.4132142689802296, 1.4132142689802296], [0.0, 1.9980019980019983], [-1.4132142689802296, -1.4132142689802296], [1.4132142689802296, 1.4132142689802296]]
[[-1.6366398918693217, 1.149522719993898], [0, 0], [1.9980019980019983, 0.0], [-1.4132142689802296, -1.4132142689802296], [1.4132142689802296, 1.4132142689802296], [-1.4132142689802296, 1.4132142689802296]]
[[-1.6368984790580192, 1.1491544627247043], [0.0, 1.9980019980019983], [-1.9980019980019983, 0.0], [-1.9980019980019983, 0.0], [0.0, -1.9980019980019983], [1.4132142689802296, 1.4132142689802296]]
[[-1.6371576494337534, 1.1487851980772439], [1.4132142689802296, 1.4132142689802296], [-1.4132142689802296, -1.4132142689802296], [0.0, 1.9980019980019983], [1.9980019980019983, 0.0], [1.4132142689802296, -1.4132142689802296]]
[[-1.6374174048916426, 1.1484149219828261], [1.4132142689802296, 1.4132142689802296], [-1.9980019980019983, 0.0], [1.9980019980

[[-1.6455089761240138, 1.1367903865969702], [-1.9980019980019983, 0.0], [-1.4132142689802296, -1.4132142689802296], [-1.4132142689802296, 1.4132142689802296], [0.0, 1.9980019980019983], [1.4132142689802296, 1.4132142689802296]]
[[-1.6448760816250358, 1.1377059564573164], [0.0, -1.9980019980019983], [1.4132142689802296, -1.4132142689802296], [1.4132142689802296, 1.4132142689802296], [-1.4132142689802296, -1.4132142689802296], [0, 0]]
[[-1.645158887877398, 1.137296966589858], [0, 0], [0.0, 1.9980019980019983], [1.4132142689802296, -1.4132142689802296], [0, 0], [1.4132142689802296, 1.4132142689802296]]
[[-1.6454423645699272, 1.136886788885427], [-1.9980019980019983, 0.0], [0, 0], [0, 0], [0.0, 1.9980019980019983], [1.4132142689802296, -1.4132142689802296]]
[[-1.645726513980969, 1.1364754182568697], [0.0, -1.9980019980019983], [1.9980019980019983, 0.0], [0, 0], [1.4132142689802296, -1.4132142689802296], [1.9980019980019983, 0.0]]
[[-1.6460113383985238, 1.1360628495884548], [-1.413214268980

KeyboardInterrupt: 

In [5]:
print("Percent of successful episodes: " + str(100.0 * sum(rList)/num_episodes) + "%")

# plt.bar(range(len(rList)), rList, color = "Blue", width = 0.00001)
# plt.show()

Percent of successful episodes: 43.2%
